### Pair Programming ETL Carga I

___

Es el momento de meter todos nuestros datos en SQL 💪🏽!!! En este ejercicio nos crearemos dos tablas en una BBDD creada por nosotras.
📌 Nota Todo lo tendremos que hacer desde jupyter notebook



In [34]:
import pandas as pd
import mysql.connector

1. Cread la BBDD con el nombre de energía. 

In [35]:
def crear_bbdd(nombre_bbdd):

    mydb = mysql.connector.connect(
      host="localhost",
      user="root",
      password="AlumnaAdalab",
      auth_plugin = 'mysql_native_password') 
    print("Conexión realizada con éxito")
    
    mycursor = mydb.cursor()

    try:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(mycursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [36]:
crear_bbdd("energy")

Conexión realizada con éxito
CMySQLCursor: CREATE DATABASE IF NOT EXISTS energy;


2. Cread las tablas de la BBDD:
- Tabla fechas
- Tabla nacional_renovable_no_renovable
- Tabla comunidades_renovable_no_renovable
- Tabla comunidades

In [37]:
def crear_insertar_tabla(nombre_bbdd, query):
    
    cnx = mysql.connector.connect(host="localhost", user="root", password="AlumnaAdalab", database=f"{nombre_bbdd}", auth_plugin = 'mysql_native_password') 

    mycursor = cnx.cursor()

    try: 
        mycursor.execute(query)
        cnx.commit() 

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [38]:
crear_tablas = """ CREATE TABLE IF NOT EXISTS `energy`.`fechas` (
                `id_date` INT NOT NULL,
                `date` DATE NOT NULL,
                PRIMARY KEY (`id_date`))
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`comunidades` (
                `id_location` INT NOT NULL,
                `location` VARCHAR(45) NOT NULL,
                PRIMARY KEY (`id_location`))
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`comunidades_renovable_no_renovable` (
                `id_location` INT NOT NULL,
                `value` FLOAT NULL,
                `percentage` FLOAT NULL,
                `energy_type` VARCHAR(45) NULL,
                `comunidades_id_location` INT NOT NULL,
                `fechas_id_date1` INT NOT NULL,
                PRIMARY KEY (`id_location`),
                INDEX `fk_comunidades_renovable_no_renovable_comunidades1_idx` (`comunidades_id_location` ASC) VISIBLE,
                INDEX `fk_comunidades_renovable_no_renovable_fechas1_idx` (`fechas_id_date1` ASC) VISIBLE,
                CONSTRAINT `fk_comunidades_renovable_no_renovable_comunidades1`
                FOREIGN KEY (`comunidades_id_location`)
                REFERENCES `energy`.`comunidades` (`id_location`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION,
                CONSTRAINT `fk_comunidades_renovable_no_renovable_fechas1`
                FOREIGN KEY (`fechas_id_date1`)
                REFERENCES `energy`.`fechas` (`id_date`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
                ENGINE = InnoDB;
                
                CREATE TABLE IF NOT EXISTS `energy`.`nacional_renovable_no_renovable` (
                `idnacional_renovable_no_renovable` INT NOT NULL,
                `value` FLOAT NULL,
                `percentage` FLOAT NULL,
                `nacional_renovable_no_renovablecol` VARCHAR(45) NULL,
                `energy_type` VARCHAR(45) NULL,
                `fechas_id_date` INT NOT NULL,
                PRIMARY KEY (`idnacional_renovable_no_renovable`),
                INDEX `fk_nacional_renovable_no_renovable_fechas1_idx` (`fechas_id_date` ASC) VISIBLE,
                CONSTRAINT `fk_nacional_renovable_no_renovable_fechas1`
                FOREIGN KEY (`fechas_id_date`)
                REFERENCES `energy`.`fechas` (`id_date`)
                ON DELETE NO ACTION
                ON UPDATE NO ACTION)
                ENGINE = InnoDB;"""

In [40]:
crear_insertar_tabla("energy", crear_tablas)

MySQLInterfaceError: Commands out of sync; you can't run this command now

Nos sale error en el commit pero, revisando Workbench, todas las tablas y foreign keys están correctamente creadas.


    
**BONUS** Insertar los datos en las tablas.
📌 Nota Esta parte del pair es optativa y no será considerada para la evaluación

In [57]:
df_energia = pd.read_csv("data/energias_años.csv", index_col=0)
df_ccaa = pd.read_csv("data/energias_comunidades.csv", index_col=0)

In [58]:
df_energia.head(2)

,value,percentage,energy_type,date
0,213821.42,0.36,Renovable,2020-01-01
1,270043.77,0.38,Renovable,2020-01-02


In [59]:
df_ccaa.head(2)

,value,percentage,energy_type,location,id_location,date
0,199198.10,1.00,No renovable,Ceuta,8744,2020-01-01
0,5602.26,0.03,Renovable,Melilla,8745,2020-01-01


Vamos a insertar en "comunidades"

In [42]:
cod_location = {"Ceuta": 8744, "Melilla": 8745, "Andalucía": 4, "Aragón": 5, "Cantabria": 6, "Castilla - La Mancha": 7, "Castilla y León": 8, "Cataluña": 9, "País Vasco": 10,
                        "Principado de Asturias": 11, "Comunidad de Madrid": 13, "Comunidad Foral de Navarra": 14, "Comunitat Valenciana": 15, "Extremadura": 16, "Galicia": 17,
                        "Illes Balears": 8743, "Canarias": 8742, "Región de Murcia": 21, "La Rioja": 20}

In [49]:
df_localidades = pd.DataFrame(pd.Series(cod_location)).reset_index()

In [56]:
df_localidades.head(2)

,index,0
0,Ceuta,8744
1,Melilla,8745


In [63]:
for indice, fila in df_localidades.iterrows():

    query_localidad = f"""
                INSERT INTO comunidades (id_location, location) 
                VALUES ( "{fila[0]}", "{fila["index"]}");"""
    
    crear_insertar_tabla("energy", query_localidad)

Vamos a insertar en "fechas"

In [70]:
fechas = (pd.concat([df_energia["date"], df_ccaa["date"]], axis = 0)).unique()

In [73]:
len(fechas)

731

In [81]:
fechas

array(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
       '2020-01-05', '2020-01-06', '2020-01-07', '2020-01-08',
       '2020-01-09', '2020-01-10', '2020-01-11', '2020-01-12',
       '2020-01-13', '2020-01-14', '2020-01-15', '2020-01-16',
       '2020-01-17', '2020-01-18', '2020-01-19', '2020-01-20',
       '2020-01-21', '2020-01-22', '2020-01-23', '2020-01-24',
       '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28',
       '2020-01-29', '2020-01-30', '2020-01-31', '2020-02-01',
       '2020-02-02', '2020-02-03', '2020-02-04', '2020-02-05',
       '2020-02-06', '2020-02-07', '2020-02-08', '2020-02-09',
       '2020-02-10', '2020-02-11', '2020-02-12', '2020-02-13',
       '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
       '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
       '2020-02-22', '2020-02-23', '2020-02-24', '2020-02-25',
       '2020-02-26', '2020-02-27', '2020-02-28', '2020-02-29',
       '2020-03-01', '2020-03-02', '2020-03-03', '2020-

In [97]:
lista = []
lista_tuplas = []
for linea in fechas:
    lista.append(linea)

for elemento in lista:
    lista_tuplas.append(tuple(elemento))

In [88]:
tuple(linea.split())

('2021-12-31',)

In [80]:
linea

'2021-12-31'

In [99]:
lista 

['2020-01-01',
 '2020-01-02',
 '2020-01-03',
 '2020-01-04',
 '2020-01-05',
 '2020-01-06',
 '2020-01-07',
 '2020-01-08',
 '2020-01-09',
 '2020-01-10',
 '2020-01-11',
 '2020-01-12',
 '2020-01-13',
 '2020-01-14',
 '2020-01-15',
 '2020-01-16',
 '2020-01-17',
 '2020-01-18',
 '2020-01-19',
 '2020-01-20',
 '2020-01-21',
 '2020-01-22',
 '2020-01-23',
 '2020-01-24',
 '2020-01-25',
 '2020-01-26',
 '2020-01-27',
 '2020-01-28',
 '2020-01-29',
 '2020-01-30',
 '2020-01-31',
 '2020-02-01',
 '2020-02-02',
 '2020-02-03',
 '2020-02-04',
 '2020-02-05',
 '2020-02-06',
 '2020-02-07',
 '2020-02-08',
 '2020-02-09',
 '2020-02-10',
 '2020-02-11',
 '2020-02-12',
 '2020-02-13',
 '2020-02-14',
 '2020-02-15',
 '2020-02-16',
 '2020-02-17',
 '2020-02-18',
 '2020-02-19',
 '2020-02-20',
 '2020-02-21',
 '2020-02-22',
 '2020-02-23',
 '2020-02-24',
 '2020-02-25',
 '2020-02-26',
 '2020-02-27',
 '2020-02-28',
 '2020-02-29',
 '2020-03-01',
 '2020-03-02',
 '2020-03-03',
 '2020-03-04',
 '2020-03-05',
 '2020-03-06',
 '2020-03-

In [100]:
lista_tuplas

[('2', '0', '2', '0', '-', '0', '1', '-', '0', '1'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '2'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '3'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '4'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '5'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '6'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '7'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '8'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '0', '9'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '0'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '1'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '2'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '3'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '4'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '5'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '6'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '7'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1', '8'),
 ('2', '0', '2', '0', '-', '0', '1', '-', '1',

In [ ]:
for indice, fila in df.iterrows():
    # creamos la query que no inserta datos en la tabla localidades
    query_localidad = f"""
                INSERT INTO localidades (latitud, longitud, ciudad)
                VALUES ( "{fila["latitud"]}", "{fila['longitud']}", "{fila['ciudad']}");
                """
    # llamamos a la funcion check_fechas para saber que fechas tenemos en la BBDD
    fechas = check_fechas("AlumnaAdalab", "energy")
    # Si la BBDD de datos esta vacía,  podemos insertar la ciudad.
    #
    if len(ciudades) == 0 or fila['ciudad'] not in ciudades[0]:
        crear_insertar_tabla("clima","AlumnaAdalab", query_localidad)
    else:
        print(f"{fila['ciudad']} ya esta en nuestra BBDD")

In [74]:
def check_fechas(contraseña, nombre_bbdd):
    cnx = mysql.connector.connect(user='root', password=f'{contraseña}',
                                  host='127.0.0.1', database=f"{nombre_bbdd}",  auth_plugin = 'mysql_native_password')
    mycursor = cnx.cursor()
    query_existe_fecha = f"""
            SELECT DISTINCT date FROM fechas
            """
    mycursor.execute(query_existe_ciudad)
    fechas = mycursor.fetchall()
    return fechas